In [3]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

HEADER = 'headless'
PAGE_LOADING_TIME = 10
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

options = webdriver.ChromeOptions()
options.add_argument(HEADER)
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
options.add_argument("lang=ko_KR")

chromedriver = '/usr/local/chromedriver/chromedriver'
driver = webdriver.Chrome(service=Service(chromedriver), options=options)

driver.get('https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000157294&dispCatNo=100000100080008&trackingCd=Cat100000100080008_Small', )

# open review page
WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#reviewInfo > a"))).click()

names, dates, reviews, stars = [], [], [], []
clicked_next = False
next_page_number = 2

while True:
    elems_page = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.pageing > a")))
    pages = []

    for elem in elems_page:
        pages.append(elem.get_attribute("data-page-no"))
    pages = list(map(int, pages))

    # if next button exists
    if pages[-1]%10 == 1:
        for page in pages:
            print("current page :", page)

            # find reviews of current page
            elems_review = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            number_of_review = len(elems_review)
            print("count the review :", number_of_review)
            
            for i in range(number_of_review):
                print("%dth review :"%i)
                name = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                name = name.text
                date = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                time.sleep(PAGE_LOADING_TIME)
                date = date.text
                review = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))
                review = review.text
                star = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.review_point > span"%(i+1)))).get_attribute("style")
                star = re.sub(r'[^0-9]', '', star)
                star = str(int(int(star) / 20))
                # time.sleep(PAGE_LOADING_TIME)
                # star = re.sub(r'[^0-9]', '',star.get_attribute("style"))
                
                # time.sleep(PAGE_LOADING_TIME)
                names.append(name)
                dates.append(date)
                reviews.append(review)
                stars.append(star)
            
            if page != pages[-1]:
                if clicked_next == True:
                    print("clicked_next True, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(next_page_number))))))
                    next_page_number += 1
                    
                elif clicked_next == False:
                    print("clicked next False, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(page))))))
                    
            elif page == pages[-1]:
                print("next page arrow click (occur document navigation)")
                driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a.next")))))
                clicked_next = True
                next_page_number = 3
                break
            
    elif pages[-1]%10 != 1:
        print("last page")
        for page in pages:
            print("current page :", page)
            
            # find reviews of current page
            elems_review = WebDriverWait(driver, 300).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.review_cont")))
            print("count the review :", number_of_review)
            
            for i in range(number_of_review):
                print("%dth review :"%i)
                name = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.info > div > p > a"%(i+1))))
                name = name.text
                date = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.date"%(i+1))))
                # time.sleep(PAGE_LOADING_TIME)
                date = date.text
                review = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.txt_inner"%(i+1))))
                review = review.text
                star = WebDriverWait(driver, 300, ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasList > li:nth-child(%d) > div.review_cont > div.score_area > span.review_point > span"%(i+1)))).get_attribute("style")
                star = re.sub(r'[^0-9]', '', star)
                star = str(int(int(star) / 20))
                # time.sleep(PAGE_LOADING_TIME)
                # star = re.sub(r'[^0-9]', '',star.get_attribute("style"))
                
                # time.sleep(PAGE_LOADING_TIME)
                names.append(name)
                dates.append(date)
                reviews.append(review)
                stars.append(star)
        
            if page != pages[-1]:
                if clicked_next == True:
                    print("clicked_next True, next page click (occur document navigation)")
                    driver.execute_script("arguments[0].click();", (WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#gdasContentsArea > div > div.pageing > a:nth-child(%d)"%(next_page_number))))))
                    next_page_number += 1
        
        if pages[-1] == page:
            break

result_dict = []
for i in range(len(names)):
    result_dict.append({
        'NAME' : names[i],
        'DATE' : dates[i],
        'REVIEW' : reviews[i],
        'STAR' : stars[i]
    })
result_df = pd.DataFrame(result_dict)
            
driver.quit()

current page : 2
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked next False, next page click (occur document navigation)
current page : 3
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked next False, next page click (occur document navigation)
current page : 4
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked next False, next page click (occur document navigation)
current page : 5
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked next False, next page click (occur document 

?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked_next True, next page click (occur document navigation)
current page : 52
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked_next True, next page click (occur document navigation)
current page : 53
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked_next True, next page click (occur document navigation)
current page : 54
count the review : 10
?th review : 0
?th review : 1
?th review : 2
?th review : 3
?th review : 4
?th review : 5
?th review : 6
?th review : 7
?th review : 8
?th review : 9
clicked_next True, next page click (occur document navigation)
current page : 55
count the review : 10
?th review : 0
?th review : 1
?th

In [10]:
names

['라먼트',
 '코덕축덕',
 '지이이잉',
 '유진찌',
 '올영은내지갑털이범',
 '러비',
 '뚜찌',
 '지르꽁',
 'jyeon****',
 '중독',
 '주또롱',
 '올영쫭쫭맨',
 '레이디킴',
 '꼬밍',
 '푸푸포푸',
 '코덕축덕',
 '나는올영잉',
 '히세',
 '수부지언니',
 'dbskdltp',
 '구름둥둥',
 'jackiek****',
 'Des1gnT0uch',
 '됴헤',
 '빛의속도',
 '이베몹',
 '잉',
 '베니샤',
 '꼼짝마움직이면서쏜다',
 '중독',
 '주또롱',
 '홍선생님',
 '시계초',
 '올영죠아',
 'Jenny17',
 'heaven****',
 'wjdwldms****',
 '쥬렁쥬렁',
 'Jenny17',
 '우앙',
 'snska****',
 '짭짤이',
 '딸ㄱi',
 '오롤롤',
 '하몽집사',
 'yeon****',
 '그루밍',
 '구름둥둥',
 '구름둥둥',
 '서누누',
 '윤민톤',
 'gkssk****',
 '뿡치',
 'rospzp',
 '으무나니',
 'ㅈㅎㅈㅎㅎ2',
 'eras****',
 '탑리뷰어메이크업',
 'dmsgml****',
 '짭짤이',
 '짭짤이',
 '뜌뜌리',
 'qkr****',
 '바나나챠챠',
 '미니언뽀라',
 '째뾰리',
 '미니언뽀라',
 '신의님',
 '중독',
 '숑이',
 '이아',
 's0nnet',
 'mde****',
 'choipur****',
 '혜즌',
 'chk****',
 '바니z',
 '설이',
 '이명준',
 '베니샤',
 'enterpri****',
 'aijin****',
 '니케',
 'dara****',
 '히솔리',
 '히히히힣',
 '영들맘',
 '파란풀',
 'dldmswjd****',
 '샘숭샘숭',
 '슈리링',
 'Loro',
 '별사탕럽',
 'joboeu****',
 'rlarjsdh****',
 '올영죠아',
 'gee9724',
 'olle',
 '말랑카카우',
 '푸르게',
 '체리22'

In [9]:
dates

['2022.05.15',
 '2021.10.21',
 '2021.05.04',
 '2021.01.31',
 '2022.03.27',
 '2021.03.01',
 '2021.02.28',
 '2020.04.05',
 '2020.03.19',
 '2022.06.23',
 '2021.07.04',
 '2020.03.28',
 '2021.03.22',
 '2022.06.22',
 '2022.03.03',
 '2022.02.28',
 '2021.12.17',
 '2021.11.24',
 '2020.03.26',
 '2020.03.18',
 '2022.02.05',
 '2021.04.15',
 '2020.03.30',
 '2020.03.30',
 '2020.03.19',
 '2020.03.19',
 '2020.03.18',
 '2022.05.13',
 '2022.02.25',
 '2021.12.06',
 '2021.10.02',
 '2021.09.20',
 '2020.12.15',
 '2020.12.13',
 '2020.07.10',
 '2020.03.21',
 '2020.03.19',
 '2020.03.18',
 '2020.03.18',
 '2020.03.18',
 '2020.03.19',
 '2022.04.11',
 '2022.03.14',
 '2022.03.07',
 '2022.02.25',
 '2021.07.25',
 '2021.07.14',
 '2021.07.04',
 '2021.07.04',
 '2021.02.10',
 '2020.10.29',
 '2020.04.04',
 '2020.03.29',
 '2020.03.21',
 '2020.03.19',
 '2021.07.16',
 '2021.06.12',
 '2020.03.19',
 '2022.06.14',
 '2022.01.03',
 '2021.12.06',
 '2021.10.01',
 '2021.08.23',
 '2021.08.01',
 '2021.06.25',
 '2021.04.06',
 '2020.12.